In [1]:
import math

#=============================================================================
#                         가스 물성치 정의
#=============================================================================

# 기본 물성치
R_universal = 8314.51  # 일반 기체 상수 [J/kmol·K]

# Air (공기)
MW_Air = 28.9660       # 분자량 [kg/kmol]
gamma_Air = 1.4020     # 비열비

# Helium (헬륨)
MW_He = 4.0026
gamma_He = 1.6670

# Hydrogen (수소)
MW_H2 = 2.0160
gamma_H2 = 1.4050

# Carbon Dioxide (이산화탄소)
MW_CO2 = 44.0100
gamma_CO2 = 1.2970


def calc_mixture_properties(X_He):
    """
    Air/He 혼합가스 물성치 계산 (몰분율 기준)
    
    Parameters:
    -----------
    X_He : float
        헬륨 몰분율 (0~1, 예: 0.5 = 50% He)
    
    Returns:
    --------
    dict : {"mw": 분자량, "gamma": 비열비}
    """
    X_Air = 1 - X_He
    
    # 혼합 분자량
    mw_mix = X_He * MW_He + X_Air * MW_Air
    
    # 질량 분율
    Y_He = (X_He * MW_He) / mw_mix
    Y_Air = 1 - Y_He
    
    # 개별 기체상수
    R_Air = R_universal / MW_Air
    R_He = R_universal / MW_He
    
    # 개별 비열
    cp_Air = gamma_Air / (gamma_Air - 1) * R_Air
    cp_He = gamma_He / (gamma_He - 1) * R_He
    cv_Air = R_Air / (gamma_Air - 1)
    cv_He = R_He / (gamma_He - 1)
    
    # 혼합 비열 (질량 평균)
    cp_mix = Y_Air * cp_Air + Y_He * cp_He
    cv_mix = Y_Air * cv_Air + Y_He * cv_He
    
    # 혼합 비열비
    gamma_mix = cp_mix / cv_mix
    
    return {"mw": mw_mix, "gamma": gamma_mix}


def get_driver_properties(driver_type, X_He=0.5):
    """
    드라이버 가스 물성치 반환
    
    Parameters:
    -----------
    driver_type : str
        "air", "he", "h2", "mix"
    X_He : float
        Air/He 혼합 시 헬륨 몰분율 (0~1)
    
    Returns:
    --------
    dict : {"mw": 분자량, "gamma": 비열비, "name": 이름}
    """
    if driver_type == "air":
        return {"mw": MW_Air, "gamma": gamma_Air, "name": "Air"}
    elif driver_type == "he":
        return {"mw": MW_He, "gamma": gamma_He, "name": "Helium"}
    elif driver_type == "h2":
        return {"mw": MW_H2, "gamma": gamma_H2, "name": "Hydrogen"}
    elif driver_type == "mix":
        props = calc_mixture_properties(X_He)
        return {"mw": props["mw"], "gamma": props["gamma"], 
                "name": f"Air/He Mix (He {X_He*100:.1f}%)"}
    else:
        raise ValueError(f"Unknown driver type: {driver_type}")


def get_driven_properties(driven_type):
    """
    드리븐 가스 물성치 반환
    
    Parameters:
    -----------
    driven_type : str
        "air", "co2"
    
    Returns:
    --------
    dict : {"mw": 분자량, "gamma": 비열비, "name": 이름}
    """
    if driven_type == "air":
        return {"mw": MW_Air, "gamma": gamma_Air, "name": "Air"}
    elif driven_type == "co2":
        return {"mw": MW_CO2, "gamma": gamma_CO2, "name": "CO2"}
    else:
        raise ValueError(f"Unknown driven type: {driven_type}")


print("물성치 정의 완료!")
print(f"  Air:  MW={MW_Air}, γ={gamma_Air}")
print(f"  He:   MW={MW_He}, γ={gamma_He}")
print(f"  H2:   MW={MW_H2}, γ={gamma_H2}")
print(f"  CO2:  MW={MW_CO2}, γ={gamma_CO2}")


물성치 정의 완료!
  Air:  MW=28.966, γ=1.402
  He:   MW=4.0026, γ=1.667
  H2:   MW=2.016, γ=1.405
  CO2:  MW=44.01, γ=1.297


In [2]:
#=============================================================================
#                         충격파 튜브 계산 함수
#=============================================================================

def calc_shock_tube(M, p1, t1, p4, t4, driven_props, driver_props):
    """
    충격파 튜브 전 상태 계산 (State 1, 2, 3, 4, 5)
    
    Parameters:
    -----------
    M : float - 충격파 마하수
    p1, t1 : float - Driven 섹션 초기 압력[Pa], 온도[K]
    p4, t4 : float - Driver 섹션 초기 압력[Pa], 온도[K]
    driven_props : dict - Driven 가스 물성치 {"mw", "gamma"}
    driver_props : dict - Driver 가스 물성치 {"mw", "gamma"}
    
    Returns:
    --------
    dict : 각 State의 물성치
    """
    # 드리븐 가스 물성치
    g1 = driven_props["gamma"]
    mw1 = driven_props["mw"]
    R1 = R_universal / mw1
    
    # 드라이버 가스 물성치
    g4 = driver_props["gamma"]
    mw4 = driver_props["mw"]
    R4 = R_universal / mw4
    
    # State 1: Driven 섹션 초기 상태
    a1 = math.sqrt(g1 * R1 * t1)
    rho1 = p1 / (R1 * t1)
    u1 = 0  # 정지 상태
    
    # 충격파 속도
    W = M * a1
    
    # State 2: 충격파 직후 (Driven 가스)
    gp1 = g1 + 1
    gm1 = g1 - 1
    
    p2_p1 = 1 + (2 * g1 / gp1) * (M**2 - 1)
    p2 = p2_p1 * p1
    
    t2_t1 = p2_p1 * ((gp1/gm1 + p2_p1) / (1 + gp1/gm1 * p2_p1))
    t2 = t2_t1 * t1
    
    rho2_rho1 = (1 + (gp1/gm1) * p2_p1) / (gp1/gm1 + p2_p1)
    rho2 = rho2_rho1 * rho1
    
    a2 = math.sqrt(g1 * R1 * t2)
    u2 = (a1 / g1) * (p2_p1 - 1) * math.sqrt((2 * g1 / gp1) / (p2_p1 + gm1/gp1))
    
    # State 3: 접촉면 (Driver 가스, p3=p2, u3=u2)
    p3 = p2
    u3 = u2
    
    # State 4: Driver 섹션 초기 상태
    a4 = math.sqrt(g4 * R4 * t4)
    rho4 = p4 / (R4 * t4)
    u4 = 0  # 정지 상태
    
    # State 3 계속: 등엔트로피 팽창파 통과 후 온도, 밀도
    gm4 = g4 - 1
    p3_p4 = p3 / p4
    t3 = t4 * (p3_p4) ** (gm4 / g4)
    rho3 = rho4 * (p3_p4) ** (1 / g4)
    a3 = math.sqrt(g4 * R4 * t3)
    
    # State 5: 반사 충격파 후 상태 (Driven 가스)
    p5_p2 = ((3*g1 - 1) * p2_p1 - gm1) / (gm1 * p2_p1 + gp1)
    p5 = p5_p2 * p2
    
    t5_t2 = p5_p2 * ((gp1/gm1 + p5_p2) / (1 + gp1/gm1 * p5_p2))
    t5 = t5_t2 * t2
    
    rho5_rho2 = (1 + (gp1/gm1) * p5_p2) / (gp1/gm1 + p5_p2)
    rho5 = rho5_rho2 * rho2
    
    a5 = math.sqrt(g1 * R1 * t5)
    u5 = 0  # 반사 후 정지
    
    return {
        "State 1 (Driven 초기)": {
            "P [Pa]": p1, "P [bar]": p1/1e5, "P [atm]": p1/101325,
            "T [K]": t1, "ρ [kg/m³]": rho1, "a [m/s]": a1, "u [m/s]": u1
        },
        "State 2 (충격파 후)": {
            "P [Pa]": p2, "P [bar]": p2/1e5, "P [atm]": p2/101325,
            "T [K]": t2, "ρ [kg/m³]": rho2, "a [m/s]": a2, "u [m/s]": u2
        },
        "State 3 (접촉면)": {
            "P [Pa]": p3, "P [bar]": p3/1e5, "P [atm]": p3/101325,
            "T [K]": t3, "ρ [kg/m³]": rho3, "a [m/s]": a3, "u [m/s]": u3
        },
        "State 4 (Driver 초기)": {
            "P [Pa]": p4, "P [bar]": p4/1e5, "P [atm]": p4/101325,
            "T [K]": t4, "ρ [kg/m³]": rho4, "a [m/s]": a4, "u [m/s]": u4
        },
        "State 5 (반사 충격파 후)": {
            "P [Pa]": p5, "P [bar]": p5/1e5, "P [atm]": p5/101325,
            "T [K]": t5, "ρ [kg/m³]": rho5, "a [m/s]": a5, "u [m/s]": u5
        },
        "Shock": {
            "Mach": M, "W [m/s]": W
        }
    }


def calc_p4_from_mach(M, p1, t1, t4, driven_props, driver_props):
    """마하수로부터 필요한 p4 계산"""
    g1 = driven_props["gamma"]
    mw1 = driven_props["mw"]
    R1 = R_universal / mw1
    
    g4 = driver_props["gamma"]
    mw4 = driver_props["mw"]
    R4 = R_universal / mw4
    
    a1 = math.sqrt(g1 * R1 * t1)
    a4 = math.sqrt(g4 * R4 * t4)
    
    gp1 = g1 + 1
    gm4 = g4 - 1
    
    p2_p1 = 1 + (2 * g1 / gp1) * (M**2 - 1)
    
    term = 1 - (gm4 * (a1/a4) * (p2_p1 - 1)) / math.sqrt(2*g1 * (2*g1 + gp1*(p2_p1 - 1)))
    p4_p1 = p2_p1 * (term ** (-2*g4/gm4))
    
    return p4_p1 * p1


def find_mach_from_p4(target_p4, p1, t1, t4, driven_props, driver_props, 
                       initial_M=3.0, tol=1e-6, max_iter=100, verbose=True):
    """
    뉴턴-랩슨 방법으로 목표 p4에서 마하수 찾기
    """
    M = initial_M
    dM = 0.001
    
    if verbose:
        print(f"목표 p4: {target_p4/1e5:.2f} bar")
        print("-" * 50)
    
    for i in range(max_iter):
        p4_calc = calc_p4_from_mach(M, p1, t1, t4, driven_props, driver_props)
        error = p4_calc - target_p4
        rel_error = abs(error / target_p4)
        
        if verbose:
            print(f"  반복 {i+1:2d}: M={M:.5f}, p4={p4_calc/1e5:.2f} bar, 오차={rel_error:.2e}")
        
        if rel_error < tol:
            if verbose:
                print(f"\n✓ 수렴! M = {M:.5f}")
            return M, True
        
        p4_plus = calc_p4_from_mach(M + dM, p1, t1, t4, driven_props, driver_props)
        dp4_dM = (p4_plus - p4_calc) / dM
        
        if abs(dp4_dM) < 1e-10:
            break
        
        M_new = M - error / dp4_dM
        M = max(1.01, min(20, M_new))
    
    if verbose:
        print(f"\n✗ 수렴 실패. INITIAL_MACH 조절 필요.")
    return M, False


print("계산 함수 정의 완료!")


계산 함수 정의 완료!


In [3]:
#=============================================================================
#                    ★★★ 입력값 설정 ★★★
#=============================================================================

#-----------------------------------------------------------------------------
# DRIVER (고압부) 설정
#-----------------------------------------------------------------------------
DRIVER_GAS = "h2"       # "air", "he", "h2", "mix" 중 선택
DRIVER_P = 110          # 압력 [bar]
DRIVER_T = 300          # 온도 [K]
DRIVER_X_He = 0.5       # Air/He 혼합 시 헬륨 몰분율 (mix일 때만 사용, 0~1)

#-----------------------------------------------------------------------------
# DRIVEN (저압부) 설정
#-----------------------------------------------------------------------------
DRIVEN_GAS = "air"      # "air", "co2" 중 선택
DRIVEN_P = 1            # 압력 [atm]
DRIVEN_T = 300          # 온도 [K]

#-----------------------------------------------------------------------------
# 뉴턴-랩슨 설정 (수렴 안 되면 조절)
#-----------------------------------------------------------------------------
INITIAL_MACH = 3.0      # 초기 추정 마하수

#=============================================================================
#                         계산 실행
#=============================================================================

# 단위 변환
p4 = DRIVER_P * 1e5      # bar → Pa
p1 = DRIVEN_P * 101325   # atm → Pa

# 물성치 가져오기
driver_props = get_driver_properties(DRIVER_GAS, DRIVER_X_He)
driven_props = get_driven_properties(DRIVEN_GAS)

print("=" * 60)
print("입력 조건")
print("=" * 60)
print(f"  Driver: {DRIVER_P} bar, {DRIVER_T} K, {driver_props['name']}")
print(f"  Driven: {DRIVEN_P} atm, {DRIVEN_T} K, {driven_props['name']}")
print("=" * 60)

# 마하수 찾기
M, converged = find_mach_from_p4(p4, p1, DRIVEN_T, DRIVER_T, 
                                  driven_props, driver_props,
                                  initial_M=INITIAL_MACH, verbose=True)

if converged:
    # 전체 상태 계산
    results = calc_shock_tube(M, p1, DRIVEN_T, p4, DRIVER_T, driven_props, driver_props)
    
    print("\n" + "=" * 60)
    print("계산 결과")
    print("=" * 60)
    
    for state, props in results.items():
        print(f"\n{state}:")
        for k, v in props.items():
            if isinstance(v, float):
                print(f"    {k}: {v:.4f}")
            else:
                print(f"    {k}: {v}")


입력 조건
  Driver: 110 bar, 300 K, Hydrogen
  Driven: 1 atm, 300 K, Air
목표 p4: 110.00 bar
--------------------------------------------------
  반복  1: M=3.00000, p4=25.13 bar, 오차=7.72e-01
  반복  2: M=6.16634, p4=385.85 bar, 오차=2.51e+00
  반복  3: M=5.22240, p4=185.69 bar, 오차=6.88e-01
  반복  4: M=4.71091, p4=122.55 bar, 오차=1.14e-01
  반복  5: M=4.58759, p4=110.56 bar, 오차=5.09e-03
  반복  6: M=4.58156, p4=110.00 bar, 오차=1.33e-05
  반복  7: M=4.58155, p4=110.00 bar, 오차=5.00e-09

✓ 수렴! M = 4.58155

계산 결과

State 1 (Driven 초기):
    P [Pa]: 101325
    P [bar]: 1.0132
    P [atm]: 1.0000
    T [K]: 300
    ρ [kg/m³]: 1.1766
    a [m/s]: 347.4631
    u [m/s]: 0

State 2 (충격파 후):
    P [Pa]: 2465867.9239
    P [bar]: 24.6587
    P [atm]: 24.3362
    T [K]: 1511.4826
    ρ [kg/m³]: 5.6835
    a [m/s]: 779.9193
    u [m/s]: 1262.3476

State 3 (접촉면):
    P [Pa]: 2465867.9239
    P [bar]: 24.6587
    P [atm]: 24.3362
    T [K]: 194.9490
    ρ [kg/m³]: 3.0669
    a [m/s]: 1062.8495
    u [m/s]: 1262.3476

State 4 

## 사용법

### 실행 순서
1. **Cell 0** 실행 (물성치 정의)
2. **Cell 1** 실행 (함수 정의)
3. **Cell 2** 실행 (입력값 설정 + 계산)

### Driver 가스 선택
| 값 | 가스 |
|------|------|
| `"air"` | Air (공기) |
| `"he"` | Helium (헬륨) |
| `"h2"` | Hydrogen (수소) |
| `"mix"` | Air/He 혼합 (DRIVER_X_He로 분율 설정) |

### Driven 가스 선택
| 값 | 가스 |
|------|------|
| `"air"` | Air (공기) |
| `"co2"` | CO2 (이산화탄소) |

### 출력 State
- **State 1**: Driven 섹션 초기 상태
- **State 2**: 충격파 통과 직후
- **State 3**: 접촉면 (팽창파 통과 후 Driver 가스)
- **State 4**: Driver 섹션 초기 상태
- **State 5**: 반사 충격파 후

### 수렴 안 될 때
`INITIAL_MACH` 값을 조절:
- p4/p1 높으면 → 더 크게 (4~6)
- p4/p1 낮으면 → 더 작게 (1.5~2)
